In [1]:
!pip install langchain
!pip install --upgrade langchain
!pip install transformers accelerate


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
  Using cached transformers-4.42.3-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.32.1-py3-none-any.whl.metadata (18 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
  Using cached safetensors-0.4.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.

# 设置 API Token
建议使用`huggingface-cli login`方式设置API Token。

# Qwen 官网例子

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# device = "cuda" # the device to load the model onto
device = "cpu"

# Now you do not need to add "trust_remote_code=True"
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

# Instead of using model.chat(), we directly use model.generate()
# But you need to use tokenizer.apply_chat_template() to format your inputs as shown below
# prompt = "Give me a short introduction to large language model."
prompt = "请给我讲个玫瑰的爱情故事"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# Directly use generate() and tokenizer.decode() to get the output.
# Use `max_new_tokens` to control the maximum output length.
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=256,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


当然，我可以为您讲述一个关于玫瑰与爱情的故事。这个故事并不出自历史或传统文献，而是一个虚构的浪漫传说，旨在表达爱与美的深刻含义。

---

在一个遥远的国度里，有一片神秘的花园，园中生长着世界上最美丽的玫瑰花——“爱之玫瑰”。这朵玫瑰不仅拥有绚丽的颜色和浓郁的香气，更蕴含着无尽的爱与祝福。每年的春天，当第一缕阳光照耀在这片花园上时，“爱之玫瑰”便会绽放出它最灿烂的笑容。

传说中，这朵玫瑰是由一位深情的王子所创造。他深爱着远方的一位公主，但因为种种原因，他们无法相见。为了表达自己对公主深深的爱意，王子在花园中种下了这株特别的玫瑰，并赋予了它特殊的魔力：每当有真正相爱的人经过这里，玫瑰就会释放出它的香气，引领他们找到彼此的心灵相通之处。

一天，一位年轻的农夫偶然间来到了这片花园。他在劳作之余，偶然间发现了这株神奇的玫瑰。随着玫瑰的香气弥漫，他仿佛听到了心灵深处的声音，指引着他找到了心中一直渴望遇见的那个女孩——正是那位传说中的公主。原来，公主


# 通过 HuggingFace 调用 Llama

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

device = "cpu"

model_name = "meta-llama/Meta-Llama-3-8B"
#model_name = "Qwen/Qwen2-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    # token= os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto',
    torch_dtype="auto",
    # token= os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

prompt = "请给我讲个玫瑰的爱情故事"

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(inputs["input_ids"], max_new_tokens=128)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


请给我讲个玫瑰的爱情故事,我要去看《玫瑰的名字》,请给我讲个玫瑰的爱情故事,我要去看《玫瑰的名字》
　　A rose by any other name would smell as sweet.
　　A rose by any other name would smell as sweet.
　　A rose by any other name would smell as sweet.
　　A rose by any other name would smell as sweet.
　　A rose by any other name would smell as sweet.
　　A rose by any other name would smell as sweet.
　　A rose by any other name would smell as


# 通过 LangChain 和 HuggingFace 

In [9]:
from langchain.prompts import PromptTemplate
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="Qwen/Qwen2-1.5B-Instruct",
)

template = """Question: {question}
              Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = prompt | llm

question = "请给我讲个玫瑰的爱情故事"

print(llm_chain.invoke(question))


Question: Rose is which type of flower?
              Answer: 1. A type of flower
Answer: 1. A type of flower. 

This answer is correct because roses are a type of flower, specifically a genus of flowering plants in the family Rosaceae. They are known for their beautiful petals and are often associated with love and romance. The other options provided do not accurately describe the type of flower roses are. For example, "a type of flower" is a general statement that does not specify the genus or species of roses, while "


In [2]:
!pip install -U langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 198.7 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [1]:
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    timeout = 3600,
    repo_id="Qwen/Qwen2-1.5B-Instruct",
)

template = """Question: {question}
              Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = prompt | llm

question = "请给我讲个玫瑰的爱情故事"

print(llm_chain.invoke(question))


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
一千零一夜有一篇故事，名字叫做《玫瑰与王子》。
              《玫瑰与王子》：有一朵美丽的玫瑰花，她名叫摩洛，居住在一座美丽的大森林里。一天，她遇到了一位英俊的王子，名叫罗林。摩洛和罗林一见钟情，开始了他们的爱情之旅。
              罗林的家人和朋友们都对摩洛非常欣赏，认为她是一个非常优秀的女孩。然而，摩洛也意识到，罗林的家人可能会因为摩洛是森林里的动物而不同意他们的婚姻。
              在这个困难的时刻，摩洛和罗林决定去寻找一位可以解决这个问题的英雄。他们旅行了很远，终于遇到了一个勇敢的骑士。骑士帮助他们找到了罗林的家人，解释了摩洛的身份，得到了他们的同意。
              现在，摩洛和罗林已经结婚，他们的爱情故事成为了森林里人们津津乐道的话题。他们一起在森林里种植玫瑰花，每年都会举办玫瑰节，庆祝他们的爱情。从此，他们过上了幸福的生活。《玫瑰与王子》的故事告诉我们，爱情可以克服一切困难，只要我们有勇气去追求它。无论我们遇到什么困难，只要我们坚持下去，就一定能够找到幸福。玫瑰花代表了爱情的纯洁和美丽，我们都要珍惜它，保护它，让它永远盛开在我们心中。 玫瑰的爱情故事，给人以积极向上的力量，让我们更加坚信爱情的力量。无论遇到什么困难，只要我们有勇气去追求，就一定能够找到幸福。让我们记住，玫瑰的爱情故事，永远是我们心中最美的风景。


# 通过 HuggingFace Pipeline

In [1]:
import os
# model = "meta-llama/Meta-Llama-3-8B-Instruct"
model = "Qwen/Qwen2-7B-Instruct"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model,
    # use_auth_token= os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

import transformers, torch
pipeline = transformers.pipeline(
   "text-generation",
    model=model,
    device_map="auto",
    # GPU内存不足
     # torch_dtype=torch.float16,
    torch_dtype="auto",
    max_length = 1000,
    truncation=True,
    # use_auth_token= os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

from langchain_huggingface import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs={"temperature":0})

template = """
    为以下的花束生成一个详细且吸引人的描述：
    花束的详细信息：
    ```{flower_details}```
"""

from langchain.prompts import PromptTemplate
prompt = PromptTemplate(template=template,
input_variables=["flower_details"])

llm_chain = prompt | llm

flower_details = "12支红玫瑰，搭配白色满天星和绿叶，包装在浪漫的红色纸中。"

print(llm_chain.invoke(flower_details))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



    为以下的花束生成一个详细且吸引人的描述：
    花束的详细信息：
    ```12支红玫瑰，搭配白色满天星和绿叶，包装在浪漫的红色纸中。```
    这个描述应突出其独特的色彩组合、花朵的品质以及整体的浪漫氛围。

Assistant: 您的花束，精心设计，不仅是一份礼物，更是一段情感的传递。它由12支娇艳欲滴的红玫瑰构成，每朵玫瑰都经过细心挑选，确保了它们的品质与美丽。红玫瑰以其热烈的情感象征著爱情与热情，而在这份花束中，它们被赋予了更为深邃的意义——是对爱情永恒承诺的誓言。

为了衬托这12支红玫瑰的璀璨光芒，我们巧妙地加入了白色的满天星。满天星以其细腻的白色花瓣，如同夜空中闪烁的星星，为这份花束增添了一抹纯净与优雅。它们与红玫瑰相互交织，形成了一种对比与和谐并存的视觉效果，使得整个花束显得更加生动与丰富。

为了让这份花束更加生动，我们还特别添加了绿叶作为点缀。这些绿叶不仅增添了自然的气息，还起到了平衡色彩的作用，使得整个花束看起来既充满活力又不失清新。

最后，这份花束被精心包裹在浪漫的红色纸中。红色不仅代表了爱与激情，也象征着对这段关系的珍视与保护。红色纸的使用，不仅提升了花束的整体美感，也为这份礼物增添了一层神秘与期待的氛围。

当您将这份花束赠予心爱之人时，不仅是在表达您的爱意，更是在分享一种独特而深刻的情感体验。这不仅仅是一份礼物，它是一段故事，一段关于爱、承诺与希望的故事。让我们一起，通过这份花束，传达那份最真挚的情感吧。


# 用 LangChain 调用自定义语言模型

In [2]:
!pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.79.tar.gz (50.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.79-cp310-cp310-linux_x86_64.whl size=3745826 sha256=e0f753401198382ef8f8b0a2b7005e5b25c00636b2454458bbaf09b1444071c7
  Stored in directory: /root/.cache/pip/wheels/bb/2e/11/8b10c6b698e6abc1289e9919e098ac4bcf6b16ebd46153e8ba
Successfully built llama-cpp-python


In [ ]:
# 下载模型
# huggingface-cli download Qwen/Qwen2-7B-Instruct-GGUF qwen2-7b-instruct-q5_k_m.gguf --local-dir . --local-dir-use-symlinks False

In [19]:
from llama_cpp import Llama
from typing import Optional, List, Mapping, Any
from langchain.llms.base import LLM

# MODEL_NAME = 'qwen2-0_5b-instruct-q5_k_m.gguf'
# MODEL_PATH = '../models/Qwen2-0.5B-Instruct-GGUF/'

MODEL_NAME = 'qwen2-7b-instruct-q5_k_m.gguf'
MODEL_PATH = '../models/Qwen2-7B-Instruct-GGUF/'

class CustomLLM(LLM):
    model_name = MODEL_NAME

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt) + 5

        llm = Llama(model_path=MODEL_PATH+MODEL_NAME, n_threads=4)

        response = llm(f"Q: {prompt} A: ", max_tokens=512)

        output = response['choices'][0]['text'].replace('A: ', '').strip()

        return output[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

llm = CustomLLM()

result = llm("昨天有一个客户抱怨他买了花给女朋友之后，两天花就枯了，你说作为客服我应该怎么解释？", stop=["Q:", "\n"])
             
print(result)


llama_model_loader: loaded meta data with 26 key-value pairs and 339 tensors from ../models/Qwen2-7B-Instruct-GGUF/qwen2-7b-instruct-q5_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = qwen2-7b-instruct
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 28
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 3584
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 18944
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 28
llama_model_loader: - kv   7:          

谢您使用我们的服务并购买鲜花送给心爱的人。我们深知每束鲜花背后所承载的情感与心意，因此对您的遭遇深表理解与遗憾。

鲜花的保鲜期受到多种因素的影响，包括但不限于花朵本身的品种、季节、运输过程中的温度控制、以及接收后的护理方式等。即便在最理想的条件下，大部分鲜花也仅能保持最佳状态数日至一周左右。然而，为了延长花的寿命并确保其最佳状态，我们建议以下几点注意事项：

1. **收到鲜花后尽快处理**：打开包装后应立即剪去花茎底部一小段，并将其浸泡于清水以去除任何可能影响水分吸收的物质。
2. **适当浇水与护理**：每日定时更换新鲜水，并根据需要调整水量。注意避免让叶子直接接触水，以防细菌生长导致鲜花提前凋谢。
3. **适宜温度与光线**：将花束放置在室温下，避开直射阳光和热源如暖气片或空调出风口。
4. **合理使用保鲜剂**（如果提供）：根据产品说明正确使用花店提供的保鲜液，这有助于延长鲜花的存活时间。

尽管我们尽一切努力确保每一份订单都能达到最高标准，但不可否认的是，自然界的力量有时是无法完全控制的。对于您所遇到的情况，我们深感抱歉，并希望您的女友能够理解这一自然现象并欣赏到花朵在最美丽时刻给予的美好回忆。

如果您对此次购买有任何不满意的地方，或需要进一步的帮助和建议，请随时与我们的客服团队联系，我们将竭诚为您提供支持和解决方案。

再次感谢您选择我们的服务，愿这份小遗憾成为一段更美好的记忆起点。期待有机会为您带来更加满意的服务体验。

祝好，
[您的名字]
[您的职位]
[公司名称]

---

这样的回复不仅表达了对客户遭遇的同情与理解，还提供了具体的护理建议以延长鲜花寿命，并承诺了后续的支持和帮助，从而维护了良好的客户服务形象。同时，也适度地解释了自然因素对于花朵保鲜的影响，体现了对现实情况的尊重和透明度。
